In [1]:
import pandas as pd
# from sqlalchemy import create_engine
import tqdm
# from config import user, password

In [42]:
import sqlite3
sqlite3.connect('../data/olympics_raw.sqlite')
df = pd.read_csv("../data/Summer Olympic medallists 1896 to 2008 - ALL MEDALISTS.csv")

In [3]:
conn = sqlite3.connect('../data/olympics.db')

In [44]:
#transform the olympic csv data
def transform_olympics(df):
    df = df.loc[3:]
    col_names = df.iloc[0].values
    df.columns = col_names
    df = df.iloc[1:]
    df = df.reset_index().drop('index', axis=1)
    return df

df = transform_olympics(df)

In [ ]:
# send cleaned olympics data to database
df.to_sql(name = 'olympics_raw', con = conn, if_exists='replace', index=False)

In [27]:
# read the data from database to check
pd.read_sql('SELECT * FROM olympics_raw LIMIT 5', conn)

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver


In [ ]:
# !pip install geopy

In [ ]:
# !pip install pycountry-convert

In [ ]:
import pycountry
import geopy

### Creating a df with total medals in each country

In [31]:
def total_medals(df):
    df = pd.get_dummies(df, columns = ['Medal'])
    df_agg = df.groupby('NOC').sum()
    df_agg['total'] = df_agg['Medal_Bronze'] + df_agg['Medal_Silver'] + df_agg['Medal_Gold']
    df_agg.columns = ['Bronze', 'Gold', 'Silver', 'Total']
    return df_agg

total_medals(df).head()

,Bronze,Gold,Silver,Total
NOC,,,,
AFG,1.0,0.0,0.0,1.0
AHO,0.0,0.0,1.0,1.0
ALG,8.0,4.0,2.0,14.0
ANZ,5.0,20.0,4.0,29.0
ARG,88.0,68.0,83.0,239.0


In [203]:
df_agg.to_sql(name = 'country_agg', con = conn, if_exists='replace', index=True)

In [211]:
temp = pd.read_sql("SELECT * FROM country_agg", con=conn)

In [212]:
temp.columns = ['ISO_A3', 'Bronze', 'Gold', 'Silver', 'Total']

ValueError: Length mismatch: Expected axis has 6 elements, new values have 5 elements

In [223]:
df_agg = df_agg[['NOC', 'Bronze', 'Gold', 'Silver', 'Total']]

In [224]:
df_agg.to_sql(name = 'country_agg', con = conn, if_exists='replace', index=False)

### Helping becky with local total json

In [225]:
df_agg = pd.read_sql("SELECT * FROM country_agg", con = conn, index_col = 'NOC')

In [226]:
import json

In [227]:
address = "../Map view/raw/countries copy.json"
with open(address) as json_file:  
    data2 = json.load(json_file)

In [228]:
agg = df_agg.reset_index()
# agg.head(10)

In [229]:
convert = pd.read_csv("../data/ISO_IOC_comparison.csv")
# convert.head(10)

In [230]:
full_table = agg.merge(convert, how='left',  left_on='NOC', right_on='IOC')
full_table.head()

,NOC,Bronze,Gold,Silver,Total,Country,IOC,FIFA,ISO
0,AFG,1.0,0.0,0.0,1.0,Afghanistan,AFG,AFG,AFG
1,AHO,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN
2,ALG,8.0,4.0,2.0,14.0,Algeria,ALG,ALG,DZA
3,ANZ,5.0,20.0,4.0,29.0,NaN,NaN,NaN,NaN
4,ARG,88.0,68.0,83.0,239.0,Argentina,ARG,ARG,ARG


In [231]:
full_table = full_table[['ISO', 'Medal_Bronze', 'Medal_Gold', 'Medal_Silver', 'total']]
full_table.head()

KeyError: "['Medal_Bronze' 'Medal_Gold' 'Medal_Silver' 'total'] not in index"

In [151]:
full_table = full_table.rename(columns={"ISO":"NOC"})
full_table.head()

,NOC,Medal_Bronze,Medal_Gold,Medal_Silver,total
0,AFG,1.0,0.0,0.0,1.0
1,NaN,0.0,0.0,1.0,1.0
2,DZA,8.0,4.0,2.0,14.0
3,NaN,5.0,20.0,4.0,29.0
4,ARG,88.0,68.0,83.0,239.0


In [153]:
full_table = full_table.set_index('NOC')
full_table.head()

,Medal_Bronze,Medal_Gold,Medal_Silver,total
NOC,,,,
AFG,1.0,0.0,0.0,1.0
NaN,0.0,0.0,1.0,1.0
DZA,8.0,4.0,2.0,14.0
NaN,5.0,20.0,4.0,29.0
ARG,88.0,68.0,83.0,239.0


In [165]:
full_table['ALG']

KeyError: 'ALG'

In [157]:
df_agg = full_table

In [159]:
for x in range(len(data2['features'])):
    temp = data2['features'][x]
    prop = temp['properties']['ISO_A3']
    try:
        print(prop)
        print(df_agg.loc[prop]['total'])
        temp['properties']['total'] = df_agg.loc[prop]['total']
        temp['properties']['gold'] = df_agg.loc[prop]['Medal_Gold']
        temp['properties']['silver'] = df_agg.loc[prop]['Medal_Silver']
        temp['properties']['bronze'] = df_agg.loc[prop]['Medal_Bronze']
    except:
        print(f"EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR")
        print(f"---------------{prop}-------------")
        print(0)
        temp['properties']['total'] = 0
        temp['properties']['gold'] = 0
        temp['properties']['silver'] = 0
        temp['properties']['bronze'] = 0

ABW
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------ABW-------------
0
AFG
1.0
AGO
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------AGO-------------
0
AIA
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------AIA-------------
0
ALB
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------ALB-------------
0
ALA
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------ALA-------------
0
AND
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------AND-------------
0
ARE
1.0
ARG
239.0
ARM
9.0
ASM
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------ASM-------------
0
ATA
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------ATA-------------
0
-99
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
----------------99-------------
0
ATF
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------ATF-------------
0
ATG
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------ATG-------------
0
AUS
1075.0
AUT
146.0
AZE
16.0
BDI
1.0
BEL
408.0
BEN
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------BEN-------------
0
BFA
EEEEEERRRRRRRROOOOOOOORRRRRRRRRRR
---------------B

In [186]:
data2['features'][66]

{'type': 'Feature',
 'properties': {'ADMIN': 'Algeria',
  'ISO_A3': 'DZA',
  'total': 14.0,
  'gold': 4.0,
  'silver': 2.0,
  'bronze': 8.0},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[8.602510428642177, 36.939510763470565],
    [8.605655151000065, 36.913045146000115],
    [8.604828329000043, 36.900978699000035],
    [8.608238973000084, 36.89072092700005],
    [8.642552124000105, 36.848501282000086],
    [8.641725301000065, 36.836357321000094],
    [8.623845255000106, 36.82607371100008],
    [8.554495483000068, 36.80364613900011],
    [8.520078979000118, 36.797910055000045],
    [8.482665242000053, 36.7999771130001],
    [8.44401127100005, 36.79620473300008],
    [8.41310876500006, 36.78390574200003],
    [8.406700887000113, 36.76798940000012],
    [8.441634155000116, 36.75320994100008],
    [8.46178796400008, 36.732797750000145],
    [8.453209676000142, 36.697657776000085],
    [8.430368693000048, 36.66262115500007],
    [8.40794112100005, 36.64262237600005],
    [8.28691491

In [163]:
count = 0
for i in data2['features']:
    print(i['properties']['total'])
    if i['properties']['total'] !=0:
        count +=1
print(count)

0
1.0
0
0
0
0
0
1.0
239.0
9.0
0
0
0
0
0
1075.0
146.0
16.0
1.0
408.0
0
0
0
331.0
0
23.0
0
0
0
92.0
0
1.0
0
372.0
1.0
0
0
0
0
592.0
376.0
33.0
679.0
1.0
0
22.0
0
0
0
0
11.0
0
0
4.0
0
395.0
0
0
0
0
41.0
1211.0
1.0
0
491.0
4.0
14.0
2.0
24.0
1.0
0
377.0
37.0
38.0
451.0
0
0
1314.0
0
0
0
1594.0
18.0
0
16.0
0
0
0
0
0
145.0
0
0
0
0
1.0
3.0
0
0
79.0
8.0
1053.0
35.0
0
178.0
0
0
24.0
48.0
1.0
17.0
7.0
1228.0
102.0
0
0
704.0
0
0
39.0
81.0
3.0
0
0
0
466.0
0
1.0
0
0
0
0
0
0
2.0
0
50.0
2.0
17.0
0
0
21.0
0
6.0
0
0
80.0
0
1.0
0
0
0
0
19.0
0
2.0
0
0
1.0
0
6.0
4.0
0
1.0
0
84.0
0
0
782.0
537.0
0
0
164.0
0
121.0
3.0
0
15.0
0
9.0
0
0
499.0
6.0
52.0
31.0
17.0
0
0
2.0
624.0
638.0
0
0
2.0
0
1.0
0
1.0
0
0
0
0
0
0
0
0
0
0
0
15.0
0
2.0
29.0
21.0
1021.0
0
0
0
3.0
0
0
1.0
21.0
2.0
0
0
1.0
0
7.0
82.0
0
42.0
2.0
6.0
148.0
0
76.0
4335.0
0
17.0
0
0
11.0
0
1.0
2.0
0
0
0
0
0
97.0
2.0
23.0
120


In [187]:
with open('geo_json_total.json', 'w') as outfile:
    json.dump(data2, outfile)

AttributeError: 'dict' object has no attribute 'head'

### Creating flags_ref file

In [ ]:
flags = pd.read_csv("../data/flag_images.csv", index_col=0)

In [ ]:
flags.head()

In [ ]:
country_ref = pd.DataFrame(df['NOC'].unique())

In [ ]:
# country_ref[0].apply(lambda x:)
# country_ref['country_name'] = ''
for i in range(len(country_ref[0])):
    try:
#         print(pycountry.countries.get(alpha_3=i).name)
        country_ref.at[i, 'country_name']= pycountry.countries.get(alpha_3=country_ref[0][i]).name
    except:
        print(country_ref[0][i])
    

In [ ]:
for i in range(country_ref.shape[0]):
    if country_ref['country_name'][i] == '':
        print(country_ref[0][i])
        c = input("Enter country name: ")
        country_ref.at[i, 'country_name']= c
        print(f"You have successfully inputted {country_ref[0][i]} as {country_ref['country_name'][i]}")
    

In [ ]:
country_ref.columns = ['code', 'country_name']

In [ ]:
country_ref.to_sql(name = 'country_ref', con = conn, if_exists='replace', index=False)

In [ ]:
pd.read_sql("SELECT * FROM country_ref", con = conn)

In [ ]:
flags.to_sql(name = 'flags_ref', con = conn, if_exists='replace', index=False)

### Haneen data asks

Is it possible to get the number of
1- male vs female for different sports for each country.  eg: in US in 2000 total gold medals for  66 males , 30 silver, female 40 gold, in 1999...etc  

Also the same as above  (per country) but with different medal types per sport, eg: in US in year 2000 for swimming women won 6 gold, 3 silver and men 4 gold and 3 silver, in year 1999....etc  

3- I was also thinking of adding a slider visualization for the  main page ( because i would only sub variables) so a filter for each sport separately through the years  grouping all medal types and genders per country per sport. Example: In 2000 for swimming: US got 33 gold (15 women, 18 men), 30 silver (20 women, 10 men), Canada got 20 (20 women) gold and 40 bronze (40 men), in year 1999....etc

In [45]:
df.head()

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver


In [56]:
han1 = df[['Edition', 'Gender', 'Discipline', 'Medal']]
pd.get_dummies(han1, columns=['Medal']).groupby(['Edition', 'Gender', 'Discipline']).sum().to_csv("haneen.csv")

In [57]:
df.nunique()

City               22
Edition            26
Sport              42
Discipline         56
Athlete         21413
NOC               138
Gender              2
Event             577
Event_gender        3
Medal               3
dtype: int64